In [1]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

# Software Optimizations -- how to write cache friendly code?

## Revisiting the design of your data structures.

### How much space does the following data structures need in physical memory?

In [2]:
compare([do_render_code("./structure/memory_usage.c", show=["//START_1","//END_1"]),do_render_code("structure/memory_usage.c", show="main")])

In [3]:
! cd structure; make clean; make memory_usage_A; ./memory_usage_A

rm -f object_of_arrays memory_usage_A memory_usage_B array_of_objects 
gcc -DHAVE_LINUX_PERF_EVENT_H -g -DA memory_usage.c -o memory_usage_A
40


Now, let's rearrange the data structure a little bit and see what's going on!

In [4]:
compare([do_render_code("./structure/memory_usage.c", show=["//START_1","//END_1"]),do_render_code("structure/memory_usage.c", show=["START_2","END_2"])])
! cd structure; make memory_usage_B; ./memory_usage_B

gcc -DHAVE_LINUX_PERF_EVENT_H -g memory_usage.c -o memory_usage_B
40


### What's a better data structure?

In [5]:
compare([do_render_code("./structure/array_of_objects.c", show=["//START","//END"]),do_render_code("structure/object_of_arrays.c", show=["//START","//END"])])

If the main workload is typically similar to 
```SELECT AVG(assignment_1) FROM table ```

In [6]:
compare([do_render_code("./structure/array_of_objects.c", show=["//START_SELECT","//END_SELECT"]),do_render_code("structure/object_of_arrays.c", show=["//START_SELECT","//END_SELECT"])])

Which one is better?

In [7]:
! cd structure; make array_of_objects; make object_of_arrays

gcc -DHAVE_LINUX_PERF_EVENT_H -g array_of_objects.c -o array_of_objects
gcc -DHAVE_LINUX_PERF_EVENT_H -g object_of_arrays.c -o object_of_arrays


In [8]:
! cd structure; echo "array of objects"; time ./array_of_objects 28800 10000 0; echo "object of arrays"; time ./object_of_arrays 28800 10000 0

array of objects

real	0m8.821s
user	0m8.396s
sys	0m0.423s
object of arrays

real	0m3.853s
user	0m3.434s
sys	0m0.417s


## Prefetch

x86 provide prefetch instructions. As a programmer, you may insert ```_mm_prefetch``` in x86 programs to perform software prefetch for your code. The gcc compiler also has a flag ```-fprefetch-loop-arrays``` to automatically insert software prefetch instructions.

### Using prefetch in matrix transpose code

The following example is a highly optimized matrix transpose code. In the example, we try to prefetch the next row.

In [9]:
render_code("./prefetch/transpose.cpp", lang="c++", show=["//START", "//END"])

// ./prefetch/transpose.cpp:42-63 (22 lines)
    //START
    //  Iterate each row.
    f_vector *y_iter = T;
    do{
        //  Iterate each column.
        f_vector *ptr_x = y_iter + block;
        f_vector *ptr_y = y_iter + row_size;

        do{

#ifdef ENABLE_PREFETCH
            _mm_prefetch((char*)(ptr_y + row_size),_MM_HINT_T0);
#endif
            swap_block(ptr_x,ptr_y,block);

            ptr_x += block;
            ptr_y += row_size;
        }while (ptr_y < stop_T);

        y_iter += iter_size;
    }while (y_iter < end);
    //END

Now, let's take a look of what's happening!

In [10]:
! cd prefetch; make clean; make
# ! echo "Without prefetch -- the baseline"; ssh htseng@celebi "lscpu | grep Model; cd courses/CS203/demo/memory/prefetch/; ./transpose"
! echo "Without prefetch -- the baseline"
! lscpu | grep Model
! ./prefetch/transpose

rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
Without prefetch -- the baseline
Model name:                           13th Gen Intel(R) Core(TM) i7-13700
Model:                                183
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.615054 seconds


In [11]:
# ! echo "With prefetch"; ssh htseng@celebi " cd courses/CS203/demo/memory/prefetch/; ./transpose_prefetch"
! echo "With prefetch"
! ./prefetch/transpose_prefetch

With prefetch
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.591875 seconds


Let's try a different machine now.

In [16]:
#! cd prefetch; make clean; make; lscpu | grep Model
#! echo "Without prefetch -- the baseline"; cd prefetch/; ./transpose
#! echo "With prefetch"; cd prefetch/; ./transpose_prefetch
! ssh htseng@xerneas "cd /nfshome/htseng/courses/CSE142/demo/memory/; make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; ssh htseng@xerneas  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose"
! echo "With prefetch";  ssh htseng@xerneas  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose_prefetch"

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
Model name:                           AMD Ryzen 9 5950X 16-Core Processor
Model:                                33
Without prefetch -- the baseline
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.53278 seconds
With prefetch
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.477222 seconds



-- It doesn't work always!

## Loop fusion/fission/interchance

In a very early lecture, we've learned the order of traversing loop matters a lot! It's actually the very first optimization in making your code cache friendly --

### Loop Interchange

In [3]:
compare([do_render_code("madd/madd_A.c",show=["//START","//END"]),do_render_code("madd/madd_B.c",show=["//START","//END"])])

Do you remember the code that let Jetson Nano suffer? What if we change it to the right hand side?

### Loop Fission

In [4]:
compare([do_render_code("loop/loop.c",show=["#else","#endif"]),do_render_code("loop/loop.c",show=["#ifdef A","#else"])])

In [29]:
! ssh htseng@nano-2 "cd courses/CSE142/demo/memory/loop; make clean; make; echo "run version A"; valgrind --tool=cachegrind ./loop_A 524288 >& nano_A.perf;  grep 'D   refs\|D1' nano_A.perf; echo "run version B"; valgrind --tool=cachegrind ./loop_B 524288 >& nano_B.perf ; grep 'D   refs\|D1' nano_B.perf"

rm -f loop_A loop_B
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  -DA perfstats.c loop.c -o loop_A
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  perfstats.c loop.c -o loop_B
run version A
==23942== D   refs:       98,362,961  (61,386,095 rd   + 36,976,866 wr)
==23942== D1  misses:      1,641,947  ( 1,051,436 rd   +    590,511 wr)
==23942== D1  miss rate:         1.7% (       1.7%     +        1.6%  )
run version B
==23944== D   refs:       98,887,245  (61,648,233 rd   + 37,239,012 wr)
==23944== D1  misses:        724,443  (   330,540 rd   +    393,903 wr)
==23944== D1  miss rate:         0.7% (       0.5%     +        1.1%  )


What if we run it on an intel processor?

In [34]:
! echo "CPU, IC, Cycles, CPI, CT, ET, miss_rate, misses, accesses" > intel_loop.csv
! make -C loop clean all;
! echo -n "Loop Fusion," >> intel_loop.csv
! ./loop/loop_A 524288 >> intel_loop.csv
! echo -n "Loop Fission," >> intel_loop.csv
! ./loop/loop_B 524288 >> intel_loop.csv
display_df_mono(render_csv("intel_loop.csv"))

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/loop'
rm -f loop_A loop_B
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  -DA perfstats.c loop.c -o loop_A
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  perfstats.c loop.c -o loop_B
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/loop'


,CPU,IC,Cycles,CPI,CT,ET,miss_rate,misses,accesses
0,Loop Fusion,2896075,2980827,1.029264,0.189545,0.000565,0.141725,186456,1315615
1,Loop Fission,4219119,3939552,0.933738,0.191138,0.000753,0.137309,253272,1844544


## Case study: matrix multiplications

GEMM that computes C = A $\times$ B is the core of many AI/ML applications. The most naive implementation of GEMM takes $O(n^3)$. Assume it takes 1 second to perform GEMM on 1,024$\times$1,024$\times$1,024 matrices. How much time do you expect it would take for 2,048$\times$2,048$\times$2,048 matrices?

In [9]:
render_code("matrix_mul/mm.c", show=["//START","//END"])

// matrix_mul/mm.c:71-92 (22 lines)
//START
void mm(double **a, double **b, double **c)
{
  int i,j,k;
  for(i = 0; i < ARRAY_SIZE; i++)
  {
    for(j = 0; j < ARRAY_SIZE; j++)
    {
      for(k = 0; k < ARRAY_SIZE; k++)
      {
        c[i][j] += a[i][k]*b[k][j];
        #ifdef DUMP
          fprintf(stderr, "a[%d][%d], %p\n",i,k, &a[i][k]);
          fprintf(stderr, "b[%d][%d], %p\n",k,j, &b[k][j]);
          fprintf(stderr, "c[%d][%d], %p\n",i,j, &c[i][j]);
        #endif
      }
    }
  }
  return;
}
//END

In [10]:
! cd matrix_mul; make clean; make mm

rm -f blockmm mm blockmm_transpose cachegrind.* mm_dump
gcc -DHAVE_LINUX_PERF_EVENT_H -O3 mm.c perfstats.c -o mm 


In [11]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > mm.csv
! ./matrix_mul/mm 32 >> ./matrix_mul/mm.csv ;./matrix_mul/mm 1024 >> ./matrix_mul/mm.csv ; ./matrix_mul/mm 2048 >> ./matrix_mul/mm.csv
#! cs203 job memory "./matrix_mul/mm 1024 >> ./matrix_mul/mm.csv ; ./matrix_mul/mm 2048 >> ./matrix_mul/mm.csv"

In [17]:
display_df_mono(render_csv("matrix_mul/mm.csv"))

,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,309118,76378,0.247084,0.667732,0.000051,nan,0,0
1,9675178441,11006852410,1.137638,0.193037,2.124731,nan,0,0
2,77382679087,116901913544,1.510699,0.192962,22.557685,nan,0,0


WOW! Compuational complexty breaks again! The GEMM performance go wild because of cache misses!

What kind of misses are we seeing?

In [17]:
! make -C matrix_mul mm_dump; ./matrix_mul/mm_dump 256 >& mm_dump_address.csv

make: Entering directory '/nfshome/htseng/courses/CS203/demo/memory/matrix_mul'
gcc -DHAVE_LINUX_PERF_EVENT_H -DDUMP -O3 mm.c perfstats.c -o mm_dump 
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/memory/matrix_mul'


In [17]:
! echo "element,address" > mm_dump_addresses_digest.csv 
! head -n 101 mm_dump_address.csv | grep "b\[" >> mm_dump_addresses_digest.csv
df = pd.read_csv("mm_dump_addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

,element,address,tag,index
0,b[0][0],0x7fb5b47b0000,0x7fb5b47b0,0x0
1,b[1][0],0x7fb5b47b0800,0x7fb5b47b0,0x20
2,b[2][0],0x7fb5b47b1000,0x7fb5b47b1,0x0
3,b[3][0],0x7fb5b47b1800,0x7fb5b47b1,0x20
4,b[4][0],0x7fb5b47b2000,0x7fb5b47b2,0x0
5,b[5][0],0x7fb5b47b2800,0x7fb5b47b2,0x20
6,b[6][0],0x7fb5b47b3000,0x7fb5b47b3,0x0
7,b[7][0],0x7fb5b47b3800,0x7fb5b47b3,0x20
8,b[8][0],0x7fb5b47b4000,0x7fb5b47b4,0x0
9,b[9][0],0x7fb5b47b4800,0x7fb5b47b4,0x20


### Matrix tiling algorithm

Let's try to partition GEMM into smaller tiles!

In [13]:
render_code("matrix_mul/blockmm.c", show=["//START","//END"])

// matrix_mul/blockmm.c:55-73 (19 lines)
//START
void blockmm(double **a, double **b, double **c)
{
  int i,j,k, ii, jj, kk;
  for(i = 0; i < ARRAY_SIZE; i+=(ARRAY_SIZE/n))
  {
    for(j = 0; j < ARRAY_SIZE; j+=(ARRAY_SIZE/n))
    {
      for(k = 0; k < ARRAY_SIZE; k+=(ARRAY_SIZE/n))
      {        
          for(ii = i; ii < i+(ARRAY_SIZE/n); ii++)
            for(jj = j; jj < j+(ARRAY_SIZE/n); jj++)
              for(kk = k; kk < k+(ARRAY_SIZE/n); kk++)
                c[ii][jj] += a[ii][kk]*b[kk][jj];
      }
    }
  }  
}
//END

In [14]:
! cd matrix_mul/; make blockmm

gcc -O3 -DHAVE_LINUX_PERF_EVENT_H blockmm.c perfstats.c -o blockmm 


In [29]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm.csv
! ./matrix_mul/blockmm 32 1 >> ./matrix_mul/blockmm.csv ;./matrix_mul/blockmm 1024 256 >> ./matrix_mul/blockmm.csv ; ./matrix_mul/blockmm 2048 256 >> ./matrix_mul/blockmm.csv; ./matrix_mul/blockmm 4096 256 >> ./matrix_mul/blockmm.csv

In [31]:
display_df_mono(render_csv("matrix_mul/blockmm.csv"))

,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,278371,114575,0.411591,0.768056,0.000088,nan,0,0
1,11847152770,3843004376,0.324382,0.193502,0.743630,nan,0,0
2,79901887120,21123990431,0.264374,0.193853,4.094950,nan,0,0
3,591324118875,158727554409,0.268427,0.193129,30.654882,nan,0,0


In [18]:
render_code("matrix_mul/blockmm_transpose.c", show=["//START","//END"])

// matrix_mul/blockmm_transpose.c:56-74 (19 lines)
//START
void blockmm_transpose(double **a, double **b, double **c)
{
  int i,j,k, ii, jj, kk;
  for(i = 0; i < ARRAY_SIZE; i+=(ARRAY_SIZE/n))
  {
    for(j = 0; j < ARRAY_SIZE; j+=(ARRAY_SIZE/n))
    {
      for(k = 0; k < ARRAY_SIZE; k+=(ARRAY_SIZE/n))
      {        
          for(ii = i; ii < i+(ARRAY_SIZE/n); ii++)
            for(jj = j; jj < j+(ARRAY_SIZE/n); jj++)
              for(kk = k; kk < k+(ARRAY_SIZE/n); kk++)
                c[ii][jj] += a[ii][kk]*b[jj][kk];
      }
    }
  }  
}
//END

### Matrix transpose

In [ ]:
! cd matrix_mul; make blockmm_transpose; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm_transpose.csv
! ./matrix_mul/blockmm_transpose 32 1 >> ./matrix_mul/blockmm_transpose.csv ;./matrix_mul/blockmm_transpose 1024 256 >> ./matrix_mul/blockmm_transpose.csv ; ./matrix_mul/blockmm_transpose 2048 256 >> ./matrix_mul/blockmm_transpose.csv; ./matrix_mul/blockmm_transpose 4096 256 >> ./matrix_mul/blockmm_transpose.csv

make: 'blockmm_transpose' is up to date.


In [ ]:
display_df_mono(render_csv("matrix_mul/blockmm_transpose.csv"))